<p style="font-weight:bold;"> <span style="font-size: 36px"> Technical Margin Allocation</span> </p>

In [0]:
#!import "../Import/Importers"
#!import "TestData"

# Workspace Initialization 

In [0]:
await Import.FromString(novelties).WithType<Novelty>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(canonicalAocTypes).WithType<AocType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(canonicalAocConfig).WithFormat(ImportFormats.AocConfiguration).WithTarget(DataSource).ExecuteAsync();

In [0]:
await DataSource.UpdateAsync<Portfolio>(dt1.RepeatOnce());
await DataSource.UpdateAsync<GroupOfContract>(new[]{dt11});
await DataSource.UpdateAsync(new[]{dt11State});
await DataSource.UpdateAsync(new[]{dt11SingleParameter});

In [0]:
await Import.FromString(amountType).WithType<AmountType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(estimateType).WithType<EstimateType>().WithTarget(DataSource).ExecuteAsync();

In [0]:
await Import.FromString(projectionConfiguration).WithType<ProjectionConfiguration>().WithTarget(DataSource).ExecuteAsync();

In [0]:
await DataSource.UpdateAsync(new [ ] {yieldCurve, yieldCurvePrevious});

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

In [0]:
await DataSource.UpdateAsync<PartitionByReportingNodeAndPeriod>(previousPeriodPartition.RepeatOnce());
await DataSource.UpdateAsync<PartitionByReportingNode>(partitionReportingNode.RepeatOnce());

# Test

In [0]:
public async Task<ActivityLog> CheckSwitchLogicAsync(IEnumerable<IfrsVariable> inputDataSet, Dictionary<AocStep,(double valueCsm, double valueLc)> csmLcSwitchBenchmark)
{
    Activity.Start();
    //Save test input data
    await Workspace.UpdateAsync<IfrsVariable>(inputDataSet);
    
    //Set up import storage and test universe
    var testStorage = new ImportStorage(previousArgs, DataSource, Workspace);
    await testStorage.InitializeAsync();
    var testUniverse = Scopes.ForStorage(testStorage).ToScope<IModel>();
    
    var identities = testUniverse.GetScopes<GetIdentities>(testStorage.DataNodesByImportScope[ImportScope.Primary].Where(dn => dn == groupOfInsuranceContracts)).SelectMany(s => s.Identities);
    var csm = testUniverse.GetScopes<AllocateTechnicalMargin>(identities, o => o.WithContext("C")).Where(x => Math.Abs(x.Value) > Precision);
    var lc = testUniverse.GetScopes<AllocateTechnicalMargin>(identities, o => o.WithContext("L")).Where(x => Math.Abs(x.Value) > Precision);
    //Clean up Workspace
    await Workspace.DeleteAsync<IfrsVariable>(await Workspace.Query<IfrsVariable>().ToArrayAsync());    
    
    var csmBenchmark = csmLcSwitchBenchmark.Where(x => Math.Abs(x.Value.valueCsm) > Precision).ToDictionary(x => x.Key, x => x.Value);
    var lcBenchmark = csmLcSwitchBenchmark.Where(x => Math.Abs(x.Value.valueLc) > Precision).ToDictionary(x => x.Key, x => x.Value);
        
        
    var errors = new List<string>();
    if(csm.Count() > csmBenchmark.Count()) 
    {
        var extraVariables = csm.Where(x => !csmBenchmark.Keys.Contains(new AocStep(x.Identity.AocType, x.Identity.Novelty)))
            .Select(x => $"AocType:{x.Identity.AocType}, Novelty:{x.Identity.Novelty}, EstimateType:{x.EstimateType}, Value:{x.Value}.");
        errors.Add( $"{nameof(AllocateTechnicalMargin)} scope for CSM has more non zero items than benchmark. Extra computed variables : \n {string.Join("\n", extraVariables)}." );
    }
    if(lc.Count() > lcBenchmark.Count()) 
    {
        var extraVariables = lc.Where(x => !lcBenchmark.Keys.Contains(new AocStep(x.Identity.AocType, x.Identity.Novelty)))
            .Select(x => $"AocType:{x.Identity.AocType}, Novelty:{x.Identity.Novelty}, EstimateType:{x.EstimateType}, Value:{x.Value}, ");
        errors.Add( $"{nameof(AllocateTechnicalMargin)} scope for LC has more non zero items compared to benchmark: \n {string.Join("\n", extraVariables)}." );
    }
    
    //Check ValueCsm
    foreach(var kvp in csmBenchmark)
    {
        var scopeSwitch = csm.SingleOrDefault(y => y.Identity.AocType == kvp.Key.AocType && y.Identity.Novelty == kvp.Key.Novelty);
        
        if(scopeSwitch == null)
        {
            errors.Add( $"Missing calculated scope for AocType and Novelty: {kvp.Key.AocType}, {kvp.Key.Novelty}." );
            continue;
        }
        
        if(Math.Abs(-1.0 * scopeSwitch.Value - kvp.Value.valueCsm) > Precision) 
            errors.Add( $"Values not matching for AocType {kvp.Key.AocType} and Novelty {kvp.Key.Novelty}. Scope: Csm {-1 * scopeSwitch.Value}; Benchmark: Csm {kvp.Value.valueCsm}." );
    }
    //Check ValueLc
    foreach(var kvp in lcBenchmark)
    {
        var scopeSwitch = lc.SingleOrDefault(y => y.Identity.AocType == kvp.Key.AocType && y.Identity.Novelty == kvp.Key.Novelty);
        
        if(scopeSwitch == null)
        {
            errors.Add( $"Missing calculated scope for AocType and Novelty: {kvp.Key.AocType}, {kvp.Key.Novelty}." );
            continue;
        }
        
        if(Math.Abs(scopeSwitch.Value - kvp.Value.valueLc) > Precision) 
            errors.Add( $"Values not matching for AocType {kvp.Key.AocType} and Novelty {kvp.Key.Novelty}. Scope: Lc {scopeSwitch.Value}; Benchmark: Lc {kvp.Value.valueLc}." );
    }
    
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
    return Activity.Finish();
}

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = previousPeriodPartition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = -10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = 15.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CL", Novelty = "C", Value = 100.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","N"),(0d, 100d)},
        {new AocStep("IA","N"), (0d,  15d)},
        {new AocStep("EV","N"), (0d, 100d)},
        {new AocStep("CL","C"), (0d, 100d)},
        {new AocStep("EA","C"), (0d, -8.0)},
        {new AocStep("AM","C"), (0d, -153.5)},
        {new AocStep("EOP","C"),(0d, 153.5)},
    };

var activity = await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = previousPeriodPartition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = -10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = 15.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CL", Novelty = "C", Value = -500.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","N"),(0d, 100d)},
        {new AocStep("IA","N"), (0d,  15d)},
        {new AocStep("EV","N"), (0d, 100d)},
        {new AocStep("CL","C"), (285d, -215d)},
        {new AocStep("EA","C"), (8d, 0d)},
        {new AocStep("AM","C"), (-146.5d, 0d)},
        {new AocStep("EOP","C"),(146.5d, 0d)},
    };

var activity = await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = previousPeriodPartition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = 100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = -10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = -150.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","N"), (0d, 100d)},
        {new AocStep("IA","N"), (50d, -100d)},
        {new AocStep("EA","C"), (8d,   0d)},
        {new AocStep("AM","C"), (-29d,   0d)},
        {new AocStep("EOP","C"),(29d,   0d)},
    };

var activity = await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
var basicIfrsVariable = new IfrsVariable{Partition = previousPeriodPartition.Id, DataNode = groupOfInsuranceContracts, AccidentYear = null, AmountType = "PR", EstimateType = "BE", EconomicBasis = "L"};
var inputDataSet = new IfrsVariable[]{
    basicIfrsVariable with {AocType = "BOP", Novelty = "I", Value = 5010.0, EstimateType = "L", AmountType = null},
    basicIfrsVariable with {AocType = "MC", Novelty = "I", Value = -10.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "I", Value = -5015.0},
    basicIfrsVariable with {AocType = "BOP", Novelty = "N", Value = -100.0},
    basicIfrsVariable with {AocType = "CF", Novelty = "N", Value = 10.0},
    basicIfrsVariable with {AocType = "IA", Novelty = "N", Value = 150.0},
    basicIfrsVariable with {AocType = "EV", Novelty = "N", Value = -45.0},
    basicIfrsVariable with {AocType = "CL", Novelty = "C", Value = -30.0},
    basicIfrsVariable with {AocType = "AM", Novelty = "C", Value = 0.5, EstimateType = "F", AmountType = null},
  };

var csmLcSwitch_benchmark = new Dictionary<AocStep,(double valueCsm, double valueLc)>()
    {
        {new AocStep("BOP","I"),(  0d, 5010.0)},
        {new AocStep("MC","I"), (  0d, -10d)},
        {new AocStep("IA","I"), (  0d, 10d)},
        {new AocStep("EV","I"), (  5d,-5010d)},
    
        {new AocStep("BOP","N"),(100d,   0d)},
        {new AocStep("IA","N"), (-100d,  50d)},
        {new AocStep("EV","N"), (  0d, -45d)},
    
        {new AocStep("CL","C"), ( 25d,  -5d)},
        {new AocStep("EA","C"), ( -8d,   0d)},
        {new AocStep("AM","C"), ( -11d,   0d)},
        {new AocStep("EOP","C"),( 11d,   0d)},
    };

var activity = await CheckSwitchLogicAsync(inputDataSet, csmLcSwitch_benchmark);
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);